In [1]:
import sys
sys.path.append("/Users/Rui/anaconda/envs/python35/lib/python3.5/site-packages")
# where the packages are located
import os, shutil, gensim, nltk,re
import numpy as np
import math
import itertools
from collections import Counter
import pandas as pd

In [2]:
def preprocess(text):
    """Given a string (text) and do 1: tokenize on spaces (remove punctuation);
    2: remove stop words (including the 2s and 3s that are from transcription);
    return the preprocessed text as string.
    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+') # define the tokenizer

    stopset = set(nltk.corpus.stopwords.words('english')).union(set(['2s', '3s',
                                                                 'um', 'yeah', 'incomprehensible',
                                                                     'incomprehesible',
                                                                     'incomprehensibe',
                                                                     'demantor',
                                                                     'laughter'])) # define set of stop words
    text = re.sub(r'\d+', '', text)
    text_ts = [w for w in tokenizer.tokenize(text) if not w in stopset]
    return(text_ts)

def text2tfDict(text):
    """Given a string and turn it to tf dictionary.i.e.
    'cigarette store need cigarette' -> {'need':0.25, 'cigarette': 0.5, 'store': 0.25}
    """
    tf_dict = dict()
    for key, value in Counter(text).items():
        tf_dict[key] = value/len(Counter(text))
        
    return(Counter(text))

def tfDict2vec(tf_dict):
    """Given a tf dictionary and calculate the weighted sum vector. shape (300,)
    return a vector named textVec
    """
    textVec = np.zeros((300,))
    for key, value in tf_dict.items():
        textVec = textVec + model[key] * value
    return(textVec)


In [ ]:
# here we import the language model (may take a while)
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [ ]:
if __name__ == "__main__":
    os.chdir("/Users/Rui/Box Sync/PSAs/05_Analyses/Rui/language_similarity/")
    data_dir = "/Users/Rui/Box Sync/PSAs/03_Data/02_psa_transcripts/transcripts/txt"

    subjs = ['PSA005', 'PSA007', 'PSA009', 'PSA010', 'PSA016', 'PSA017', 'PSA019',
         'PSA026', 'PSA027', 'PSA028', 'PSA029', 'PSA030', 'PSA032', 'PSA036',
         'PSA038', 'PSA056', 'PSA061', 'PSA068', 'PSA069', 'PSA078', 'PSA079',
         'PSA086', 'PSA094', 'PSA108', 'PSA109', 'PSA112', 'PSA119', 'PSA120',
         'PSA127', 'PSA128', 'PSA131', 'PSA132', 'PSA142','PSA144', 'PSA153',
         'PSA154', 'PSA156'] #37 subjs
    
    test_num = 5

    # create a dataframe
    df = pd.DataFrame(np.nan, index = range(15904), columns = ['subj1', 'vID1', 'subj2', 'vID2',
                   'language_sim', 'txt1','txt2'])
    rownum = 0
    for i in list(itertools.combinations(range(test_num),2)):
        print(i)
        # two different participants talking about different ads
        for j in list(itertools.combinations(range(12),2)):
            df['subj1'][rownum] = subjs[i[0]]
            df['vID1'][rownum] = j[0] + 1
            df['subj2'][rownum] = subjs[i[1]]
            df['vID2'][rownum] = j[1] + 1

            # read in two texts to compare
            f1_open = open(os.path.join(data_dir, subjs[i[0]] + "_" + str(j[0] + 1) + ".txt"), 'rb')
            txt1 = f1_open.read().decode('utf8', 'ignore')
            f1_open.close()

            f2_open = open(os.path.join(data_dir, subjs[i[1]] + "_" + str(j[1] + 1) + ".txt"), 'rb')
            txt2 = f2_open.read().decode('utf8', 'ignore')
            f2_open.close()

            df['txt1'][rownum] = txt1
            df['txt2'][rownum] = txt2

            # get the vec for txt1
            txt1_ts = preprocess(txt1)
            txt1_tf = text2tfDict(txt1_ts)
            txt1_vec = tfDict2vec(txt1_tf)

            # get the vec for txt2
            txt2_ts = preprocess(txt2)
            txt2_tf = text2tfDict(txt2_ts)
            txt2_vec = tfDict2vec(txt2_tf)

            # calculate the euclidian distance between two vectors

            df['language_sim'][rownum] = np.corrcoef(txt1_vec, txt2_vec)[0,1]

            rownum += 1
        # two different participants talking about the same ads
        for k in range(12):
            df['subj1'][rownum] = subjs[i[0]]
            df['vID1'][rownum] = k + 1
            df['subj2'][rownum] = subjs[i[1]]
            df['vID2'][rownum] = k + 1

            # read in two texts to compare
            f1_open = open(os.path.join(data_dir, subjs[i[0]] + "_" + str(k + 1) + ".txt"), 'rb')
            txt1 = f1_open.read().decode('utf8', 'ignore')
            f1_open.close()

            f2_open = open(os.path.join(data_dir, subjs[i[1]] + "_" + str(k + 1) + ".txt"), 'rb')
            txt2 = f2_open.read().decode('utf8', 'ignore')
            f2_open.close()

            df['txt1'][rownum] = txt1
            df['txt2'][rownum] = txt2

            # get the vec for txt1
            txt1_ts = preprocess(txt1)
            txt1_tf = text2tfDict(txt1_ts)
            txt1_vec = tfDict2vec(txt1_tf)

            # get the vec for txt2
            txt2_ts = preprocess(txt2)
            txt2_tf = text2tfDict(txt2_ts)
            txt2_vec = tfDict2vec(txt2_tf)

            # calculate the euclidian distance between two vectors

            df['language_sim'][rownum] = np.corrcoef(txt1_vec, txt2_vec)[0,1]

            rownum += 1
            
    # same participant talking about different ads
    for i in range(test_num):
        print(i)
        for j in list(itertools.combinations(range(12),2)):
            df['subj1'][rownum] = subjs[i]
            df['vID1'][rownum] = j[0] + 1
            df['subj2'][rownum] = subjs[i]
            df['vID2'][rownum] = j[1] + 1

            # read in two texts to compare
            f1_open = open(os.path.join(data_dir, subjs[i] + "_" + str(j[0] + 1) + ".txt"), 'rb')
            txt1 = f1_open.read().decode('utf8', 'ignore')
            f1_open.close()

            f2_open = open(os.path.join(data_dir, subjs[i] + "_" + str(j[1] + 1) + ".txt"), 'rb')
            txt2 = f2_open.read().decode('utf8', 'ignore')
            f2_open.close()

            df['txt1'][rownum] = txt1
            df['txt2'][rownum] = txt2

            # get the vec for txt1
            txt1_ts = preprocess(txt1)
            txt1_tf = text2tfDict(txt1_ts)
            txt1_vec = tfDict2vec(txt1_tf)

            # get the vec for txt2
            txt2_ts = preprocess(txt2)
            txt2_tf = text2tfDict(txt2_ts)
            txt2_vec = tfDict2vec(txt2_tf)

            # calculate the euclidian distance between two vectors

            df['language_sim'][rownum] = np.corrcoef(txt1_vec, txt2_vec)[0,1]

            rownum += 1
            
        

df.to_csv("/Users/Rui/Desktop/language_dis.csv")

(0, 1)


/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rui/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWit

(0, 2)
(0, 3)
(0, 4)
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)
0


/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a 

1
2
3
4


In [ ]:
if __name__ == "__main__":
    os.chdir("/Users/Rui/Box Sync/PSAs/05_Analyses/Rui/language_similarity/")
    data_dir = "/Users/Rui/Box Sync/PSAs/03_Data/02_psa_transcripts/transcripts/txt"

    subjs = ['PSA005', 'PSA007', 'PSA009', 'PSA010', 'PSA016', 'PSA017', 'PSA019',
         'PSA026', 'PSA027', 'PSA028', 'PSA029', 'PSA030', 'PSA032', 'PSA036',
         'PSA038', 'PSA056', 'PSA061', 'PSA068', 'PSA069', 'PSA078', 'PSA079',
         'PSA086', 'PSA094', 'PSA108', 'PSA109', 'PSA112', 'PSA119', 'PSA120',
         'PSA127', 'PSA128', 'PSA131', 'PSA132', 'PSA142','PSA144', 'PSA153',
         'PSA154', 'PSA156'] #37 subjs

    # create a dataframe
    #df = pd.DataFrame(np.nan, index = range(7992), columns = ['subj1', 'subj2', 'vID',
    #               'language_dis'])
    #rownum = 0
    #for i in list(itertools.combinations(range(37),2)):
    #    for j in range(12):
    #        print(rownum)
    #        df['subj1'][rownum] = i[0]
    #        df['subj2'][rownum] = i[1]
    #        df['vID'][rownum] = j + 1

            # read in two texts to compare
    for i in (range(12)):
        f1_open = open(os.path.join(data_dir, subjs[1] + "_" + str(1) + ".txt"), 'rb')
        txt1 = f1_open.read().decode('utf8', 'ignore')
        f1_open.close()

        f2_open = open(os.path.join(data_dir, subjs[1] + "_" + str(i + 1) + ".txt"), 'rb')
        txt2 = f2_open.read().decode('utf8', 'ignore')
        f2_open.close()

        # get the vec for txt1
        txt1_ts = preprocess(txt1)
        txt1_tf = text2tfDict(txt1_ts)
        txt1_vec = tfDict2vec(txt1_tf)

        # get the vec for txt2
        txt2_ts = preprocess(txt2)
        txt2_tf = text2tfDict(txt2_ts)
        txt2_vec = tfDict2vec(txt2_tf)

        print(txt1)
        print(i + 1)
        print(txt2)


        plt.plot(txt1_vec)
        plt.plot(txt2_vec)
        plt.ylabel('word2vec')
        plt.show()

        print(np.corrcoef(txt1_vec, txt2_vec))